In [1]:
%pip install -q langchain chromadb loguru openai langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import openai

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
import langchain
from langchain.tools import tool
from langchain.llms import OpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.runnables.base import RunnableSequence
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.runnables.utils import Output
from langchain_core.messages import BaseMessage
from langchain.schema import AgentFinish
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.llms.openai import BaseOpenAI
import langchain_core

from typing import List

import pydantic
from loguru import logger
from rich.pretty import pprint

## Tools


In [4]:
@tool
def sum(a: int, b: int)->str:
    """Perform the addition of variables a and b, i.e., calculate a + b."""
    return f"The result of {a} + {b} is {str(a+b)}"

@tool
def random_joke()->str:
    """A crazy creator that can create a random joke."""
    model: BaseOpenAI = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=1.5)
    res = model.predict(f"You are a crazy creator that can create a random joke.")
    return res.strip()

@tool
def translate(string: str)->str:
    """Translate the string into Chinese."""
    model: BaseOpenAI = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)
    res = model.predict(f"Translate {string} into Chinese, only return the result.")
    return res.strip()

In [5]:
args = {
    "a":1, "b":2
}
sum.run(args)

args = {
    "string":"hello,world"
}
translate.run(args)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'你好，世界。'

## Agent

### Convert tools into OpenAI functions(tools)

In [6]:
functions = [
    sum,
    translate
]

In [7]:
pprint(functions)

[
│   StructuredTool(
│   │   name='sum',
│   │   description='sum(a: int, b: int) -> str - Perform the addition of variables a and b, i.e., calculate a + b.',
│   │   args_schema=<class 'pydantic.main.sumSchemaSchema'>,
│   │   func=<function sum at 0x7e92771ae950>
│   ),
│   StructuredTool(
│   │   name='translate',
│   │   description='translate(string: str) -> str - Translate the string into Chinese.',
│   │   args_schema=<class 'pydantic.main.translateSchemaSchema'>,
│   │   func=<function translate at 0x7e92a82655a0>
│   )
]

### Prompt

In [8]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are an assisant to perform the user input."),
    HumanMessagePromptTemplate.from_template("{input}"),
])

### Require chat model

In [9]:
model: BaseChatModel = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0).bind_functions(functions=functions)

#### The OpenAI model fires exception.

`TypeError: Completions.create() got an unexpected keyword argument 'functions'`



```python
prompt = PromptTemplate.from_template("""
You are an assisant to perform the user input:

{input}
""")
model: BaseOpenAI = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=1.5).bind_.bind_functions(functions=functions)
```



#### Model responses `function_call`

It determines which function(tool) to call and which arguments to use.

In [10]:
chain: RunnableSequence = prompt | model
res: Output = chain.invoke({"input":"""
Translate the following sentence:

Hello,world
"""})
pprint(res)

AIMessage(
│   content='',
│   additional_kwargs={'function_call': {'arguments': '{\n  "string": "Hello, world"\n}', 'name': 'translate'}}
)

#### Key: `OpenAIFunctionsAgentOutputParser`

Converts the OpenAI function_call response into equivalent Python objects.

For example, the outcome is `AgentActionMessageLog`, detailing the `tool` to be utilized and `tool_input`: the dictionary that the specified `tool` accepts,

so:

```
tool.run(tool_input)
```

#### Parse `AIMessage`  with  `function_call` in `additional_kwargs`.

In [11]:
tool_exec_chain: RunnableSequence = chain | OpenAIFunctionsAgentOutputParser()
result: Output = tool_exec_chain.invoke({"input":"""
Translate the following sentence:

Hallo,Welt, Hello,world
"""})

pprint(result)

AgentActionMessageLog(
│   tool='translate',
│   tool_input={'string': 'Hallo,Welt, Hello,world'},
│   log="\nInvoking: `translate` with `{'string': 'Hallo,Welt, Hello,world'}`\n\n\n",
│   message_log=[
│   │   AIMessage(
│   │   │   content='',
│   │   │   additional_kwargs={
│   │   │   │   'function_call': {
│   │   │   │   │   'arguments': '{\n  "string": "Hallo,Welt, Hello,world"\n}',
│   │   │   │   │   'name': 'translate'
│   │   │   │   }
│   │   │   }
│   │   )
│   ]
)

#### `AgentActionMessageLog`, if one function(tool) is required

In [12]:
tools={
            "sum":sum,
            "translate":translate,
        }

In [13]:
tools[result.tool].run(result.tool_input)

'你好，世界'

In [14]:
result: Output = tool_exec_chain.invoke({"input":"""
What is 1 plus 1?
"""})
pprint(result)

AgentActionMessageLog(
│   tool='sum',
│   tool_input={'a': 1, 'b': 1},
│   log="\nInvoking: `sum` with `{'a': 1, 'b': 1}`\n\n\n",
│   message_log=[
│   │   AIMessage(
│   │   │   content='',
│   │   │   additional_kwargs={'function_call': {'arguments': '{\n  "a": 1,\n  "b": 1\n}', 'name': 'sum'}}
│   │   )
│   ]
)

In [15]:
result.tool, result.tool_input

('sum', {'a': 1, 'b': 1})

In [16]:
tools[result.tool].run(result.tool_input)

'The result of 1 + 1 is 2'

#### `AgentFinish`, if no function (tool) is required.

In [17]:
chain: RunnableSequence = prompt | model
res: Output = chain.invoke({"input":"""
hi
"""})
pprint(res)

AIMessage(content='Hello! How can I assist you today?')

In [18]:
tool_exec_chain: RunnableSequence = chain | OpenAIFunctionsAgentOutputParser()
result: Output = tool_exec_chain.invoke({"input":"""
Hi
"""})

pprint(result)

AgentFinish(
│   return_values={'output': 'Hello! How can I assist you today?'},
│   log='Hello! How can I assist you today?'
)

### Simple Agent, one interaction

When a specific tool is requested by an OpenAI response, the process typically does not proceed to `langchain.schema.AgentFinish`. This implies that whenever operations like `sum` or `translate` are required, `langchain.schema.AgentAction` is invoked instead.

#### Only request one task

In [19]:
def agent_resolver(res: Output):
    logger.info(type(res))

    if isinstance(res, AgentFinish):
        return res.return_values["output"]
    else:
        tools={
            "sum":sum,
            "translate":translate,
        }
        return tools[res.tool].run(res.tool_input)

In [20]:
agent_chain: RunnableSequence = tool_exec_chain | agent_resolver

In [21]:
result: Output = agent_chain.invoke({"input":"""
Translate the following sentence:

Hallo,Welt, Hello,world
"""})
pprint(result)

2024-02-06 08:28:47.099 | INFO     | __main__:agent_resolver:2 - <class 'langchain_core.agents.AgentActionMessageLog'>


'你好，世界'

In [22]:
result: Output = agent_chain.invoke({"input":"""
What is the 1 plus 1 ?
"""})
pprint(result)

2024-02-06 08:28:48.361 | INFO     | __main__:agent_resolver:2 - <class 'langchain_core.agents.AgentActionMessageLog'>


'The result of 1 + 1 is 2'

In [23]:
result: Output = agent_chain.invoke({"input":"""
Hello
"""})
pprint(result)

2024-02-06 08:28:48.775 | INFO     | __main__:agent_resolver:2 - <class 'langchain_core.agents.AgentFinish'>


'Hi there! How can I assist you today?'

#### Request two tasks, not fully worked

1. The `sum()` function executed successfully.
2. The `translate()` function failed because the agent exited peacefully after the completion of `sum()`.


In [24]:
result: Output = agent_chain.invoke({"input":"""
What is the 1 plus 1, answer me with a translated result?
"""})
pprint(result)

2024-02-06 08:28:49.610 | INFO     | __main__:agent_resolver:2 - <class 'langchain_core.agents.AgentActionMessageLog'>


'The result of 1 + 1 is 2'

### Agent loop

Start a new example.

Use `random_joke()` to generate a crazy joke and translate the response.

It is a continuous loop that executes the `chain` until the `AgentFinish` condition is met, without consideration for any errors or infinite loop exceptions.

#### Key: Redefine the prompt to record history , allowing for the subsequent execution of the `translate()` function after obtaining the `random_joke()` result.

In [25]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are an assisant to perform the user input."),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="history"),
])

In [26]:
functions = [
    random_joke,
    translate
]

In [27]:
pprint(functions)

[
│   StructuredTool(
│   │   name='random_joke',
│   │   description='random_joke() -> str - A crazy creator that can create a random joke.',
│   │   args_schema=<class 'pydantic.main.random_jokeSchemaSchema'>,
│   │   func=<function random_joke at 0x7e92a8266ef0>
│   ),
│   StructuredTool(
│   │   name='translate',
│   │   description='translate(string: str) -> str - Translate the string into Chinese.',
│   │   args_schema=<class 'pydantic.main.translateSchemaSchema'>,
│   │   func=<function translate at 0x7e92a82655a0>
│   )
]

In [28]:
model: BaseChatModel = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=1.5).bind_functions(functions=functions)

In [29]:
# The last node is not some like agent_resolver() above, because we`ll use loop the whole agent logical
# in it.
agent_chain: RunnableSequence = prompt | model | OpenAIFunctionsAgentOutputParser()

In [30]:
def agent_executor(agent_chain: RunnableSequence, human_input: str) -> Output:
    history = list()

    tools = {
        "random_joke":random_joke,
        "translate":translate,
    }

    while True:
        fmt_openai_funs: List[BaseMessage] = format_to_openai_functions(history)
        pprint("format_to_openai_functions:")
        pprint(fmt_openai_funs)
        pprint("---------------")

        res: Output = agent_chain.invoke({"input": human_input,
                                        "history": fmt_openai_funs})
        logger.debug(f"Type of chain res: {type(res)}")
        logger.debug(res)

        if isinstance(res, AgentFinish):
            pprint("history list:")
            pprint(history)
            pprint("---------------")

            return res
        else:
            tool_res = tools[res.tool].run(res.tool_input)
            logger.debug(f"Type of tool res: {type(tool_res)}")
            logger.debug(f"Tool: {res.tool} returned: {tool_res}")
            history.append((res, tool_res))

In [31]:
result: Output = agent_executor(agent_chain=agent_chain,
                                human_input="""
Please create a random joke and provide the translated result
without including the original language, newlines, empty spaces, or instructions.
""")

'format_to_openai_functions:'

[]

'---------------'

2024-02-06 08:28:50.643 | DEBUG    | __main__:agent_executor:17 - Type of chain res: <class 'langchain_core.agents.AgentActionMessageLog'>
2024-02-06 08:28:50.645 | DEBUG    | __main__:agent_executor:18 - tool='random_joke' tool_input={} log='\nInvoking: `random_joke` with `{}`\n\n\n' message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{}', 'name': 'random_joke'}})]
2024-02-06 08:28:51.210 | DEBUG    | __main__:agent_executor:28 - Type of tool res: <class 'str'>
2024-02-06 08:28:51.211 | DEBUG    | __main__:agent_executor:29 - Tool: random_joke returned: Okay, here it goes:

"Why did the scarecrow win an award?

Because he was outstanding in his field!"


'format_to_openai_functions:'

...

'---------------'

2024-02-06 08:28:53.151 | DEBUG    | __main__:agent_executor:17 - Type of chain res: <class 'langchain_core.agents.AgentFinish'>
2024-02-06 08:28:53.154 | DEBUG    | __main__:agent_executor:18 - return_values={'output': '为什么稻草人赢得了奖项？因为他在田地里表现出色！'} log='为什么稻草人赢得了奖项？因为他在田地里表现出色！'


'history list:'

[
│   (
│   │   AgentActionMessageLog(
│   │   │   tool='random_joke',
│   │   │   tool_input={},
│   │   │   log='\nInvoking: `random_joke` with `{}`\n\n\n',
│   │   │   message_log=[
│   │   │   │   AIMessage(
│   │   │   │   │   content='',
│   │   │   │   │   additional_kwargs={'function_call': {'arguments': '{}', 'name': 'random_joke'}}
│   │   │   │   )
│   │   │   ]
│   │   ),
│   │   'Okay, here it goes:\n\n"Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!"'
│   ),
│   (
│   │   AgentActionMessageLog(
│   │   │   tool='translate',
│   │   │   tool_input={'string': 'Why did the scarecrow win an award? Because he was outstanding in his field!'},
│   │   │   log="\nInvoking: `translate` with `{'string': 'Why did the scarecrow win an award? Because he was outstanding in his field!'}`\n\n\n",
│   │   │   message_log=[
│   │   │   │   AIMessage(
│   │   │   │   │   content='',
│   │   │   │   │   additional_kwargs={
│   │   │   │   │   │   'function_call': {
│   │   │   │   │   │   │   'arguments': '{\n  "string": "Why did the scarecrow win an award? Because he was outstanding in his field!"\n}',
│   │   │   │   │   │   │   'name': 'translate'
│   │   │   │   │   │   }
│   │   │   │   │   }
│   │   │   │   )
│   │   │   ]
│   │   ),
│   │   '为什么稻草人赢得了奖项？因为他在田地里表现出色！'
│   )
]

'---------------'

In [32]:
pprint(result)

AgentFinish(
│   return_values={'output': '为什么稻草人赢得了奖项？因为他在田地里表现出色！'},
│   log='为什么稻草人赢得了奖项？因为他在田地里表现出色！'
)